In [1]:
%use fuel(2.3.1)

In [2]:
import java.math.BigInteger

In [3]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [4]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 8

In [5]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [7]:
val sample = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

In [8]:
val input = getInput(year, day, session)

In [9]:
fun cross(s: String, t: String): Int {
    return s.toSet().intersect(t.toSet()).size
}

fun norm(s: String): String {
    val t = s.toMutableList()
    t.sort()
    return t.joinToString("")
}

fun decode(line: String): List<Int> {
    val (dict, msg) = line.split('|')
    val lw = dict
        .split(' ')
        .filter { it.length > 0 }
    val code = Array(10) { "" }
    
    for (word in lw) {
        when (word.length) {
            2 -> code[1] = norm(word)
            3 -> code[7] = norm(word)
            4 -> code[4] = norm(word)
            7 -> code[8] = norm(word)
            else -> {}
        }
    }
    
    for (word in lw) {
        if (word.length == 5 && cross(word, code[4]) == 2) {
            code[2] = norm(word)
        }
    }
    
    for (word in lw) {
        if (word.length == 5) {
            when (cross(word, code[2])) {
                3 -> code[5] = norm(word)
                4 -> code[3] = norm(word)
                else -> {}
            }
        }
    }
    
    for (word in lw) {
        if (word.length == 6) {
            if (cross(word, code[3]) == 5) {
                code[9] = norm(word)
            } else if (cross(word, code[1]) == 2) {
                code[0] = norm(word)
            } else {
                code[6] = norm(word)
            }
        }
    }
    
    val mapping = (0 until 10).map {
        code[it] to it
    }.toMap()
    
    return msg
        .split(' ')
        .filter { it.length > 0 }
        .map { mapping.getOrDefault(norm(it), 0) }
}

In [10]:
fun partOne(input: String): String {
    val lines = input.split('\n').filter {it.length > 0}
    return lines.map {
        decode(it).count {x -> x in setOf(1, 4, 7, 8)}
    }.sum().toString()
}

In [11]:
partOne(sample)

26

In [12]:
val partOneAns = partOne(input)
partOneAns

519

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [13]:
fun partTwo(input: String): String {
    val lines = input.split('\n').filter {it.length > 0}
    return lines.map {
        decode(it)
            .map { it.toString() }
            .joinToString("")
            .toInt()
    }.sum().toString()
}

In [14]:
partTwo(sample)

61229

In [15]:
val partTwoAns = partTwo(input)
partTwoAns

1027483

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)